# Noise mix

See `~/data/2023_mix_chr1` folder.

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format='retina'

import pandas as pd
import seaborn as sns
from IPython.display import display
from tqdm.auto import tqdm

sns.set_style("whitegrid")
import matplotlib.pyplot as plt
import numpy as np
import os
import re
from itertools import product
import tempfile

In [ ]:
def bedl(file):
    try:
        tf = pd.read_csv(file, sep='\t', header=None)
        return tf[2] - tf[1]
    except:
        return np.zeros(0)  # Empty file


def lines(file):
    try:
        tf = pd.read_csv(file, sep='\t', header=None)
        return len(tf)
    except:
        return 0  # Empty file


def d(a, b):
    return a / b if b != 0 else 0


def last_col(file):
    try:
        cols = len(pd.read_csv(file, sep='\t', nrows=1, header=None).columns)
        return pd.read_csv(file, sep='\t', header=None, usecols=[cols - 1])[cols - 1]
    except:
        return np.zeros(0)  # Empty file


def sorted_file(file):
    ts = tempfile.mktemp()
    ! cat {file} | sort -k1,1 -k2,2n -k3,3n > {ts}
    return ts

# Load hg38 peaks


In [ ]:
GSE26320_PATH = os.path.expanduser('~/data/2023_GSE26320')
GSE26320_CELLS = ['GM12878', 'HMEC', 'HSMM', 'K562', 'NHEK', 'NHLF', 'H1', 'Huvec', 'HepG2']
MODIFICATIONS = ['H3K4me3', 'H3K27ac', 'H3K4me1', 'H3K27me3', 'H3K36me3']
GSE26320_REPS = ['rep1', 'rep2']

IMMUNE_PATH = os.path.expanduser('~/data/2023_Immune')
IMMUNE_CELLS = ['CD4ABT', 'TCellBB', 'BCell', 'TCell', 'Monocyte', 'PBMC', 'NK', 'CD34', 'CD4', ]  ## Longest first
IMMUNE_REPS = ['rep1', 'rep2', 'rep3', '']

! mkdir -p {GSE26320_PATH}/pics
! mkdir -p {IMMUNE_PATH}/pics

In [ ]:
TOOLS = ['MACS2', 'MACS2 broad', 'SPAN', 'SICER']
palette = plt.cm.get_cmap('tab10')
TOOLS_PALETTE = {t: palette(i) for i, t in enumerate(TOOLS)}

In [ ]:
def load_peaks(path, suffix, modifications, cells, replicates):
    df = pd.DataFrame(columns=['file', 'modification', 'cell', 'replicate', 'peaks'],
                      dtype=object)
    for f in tqdm(os.listdir(path)):
        if not f.endswith(suffix):
            continue
        cell = next((c for c in cells if c in f), None)
        mod = next((m for m in modifications if m in f), None)
        rep = next((r for r in replicates if r in f), None)
        if cell and mod and rep is not None:
            peaks_path = os.path.join(path, f)
            ps = lines(peaks_path)
            df.loc[len(df)] = (peaks_path, mod, cell, rep, ps)
    return df

In [ ]:
def load_peaks_path(path, modifications, cells, replicates):
    df_macs2 = load_peaks(os.path.join(path, 'macs2'), '.narrowPeak', modifications, cells, replicates)
    df_macs2['tool'] = 'MACS2'
    print('MACS2', len(df_macs2))

    df_macs2broad = load_peaks(os.path.join(path, 'macs2'), '.broadPeak', modifications, cells, replicates)
    df_macs2broad['tool'] = 'MACS2 broad'
    print('MACS2 broad', len(df_macs2broad))

    df_sicer = load_peaks(os.path.join(path, 'sicer'), 'summary-FDR0.01', modifications, cells, replicates)
    df_sicer['tool'] = 'SICER'
    print('SICER', len(df_sicer))

    df_span = load_peaks(os.path.join(path, 'span'), '.peak', modifications, cells, replicates)
    df_span['tool'] = 'SPAN'
    print('SPAN', len(df_span))

    return pd.concat([df_macs2, df_macs2broad, df_sicer, df_span]).reset_index(drop=True)

In [ ]:
df_encode_peaks = load_peaks_path(GSE26320_PATH, MODIFICATIONS, GSE26320_CELLS, GSE26320_REPS)
df_encode_peaks['dataset'] = 'ENCODE'
df_encode_peaks.sample(5)

In [ ]:
df_immune_peaks = load_peaks_path(IMMUNE_PATH, MODIFICATIONS, IMMUNE_CELLS, IMMUNE_REPS)
df_immune_peaks['dataset'] = 'Roadmap'
df_immune_peaks.sample(5)

In [ ]:
df_peaks = pd.concat([df_encode_peaks, df_immune_peaks]).reset_index(drop=True)

In [ ]:
# Remove outliers, see 2023 - figures.ipynb
OUTLIERS = [('H3K4me3', 'NK', ''),
            ('H3K4me1', 'NK', ''),
            ('H3K27me3', 'TCell', ''),
            ('H3K27me3', 'NHLF', 'rep2'),
            ('H3K27me3', 'GM12878', 'rep1')]
df_peaks = df_peaks.loc[[(m, c, r) not in OUTLIERS
                         for _, (m, c, r) in df_peaks[['modification', 'cell', 'replicate']].iterrows()]]

In [ ]:
plt.figure(figsize=(14, 4))
axs = [plt.subplot(1, 5, i + 1) for i in range(5)]
for i, m in enumerate(MODIFICATIONS):
    ax = axs[i]
    sns.barplot(data=df_peaks[df_peaks['modification'] == m], x='tool', y='peaks', hue='dataset',
                capsize=.2, err_kws={'linewidth': 2}, edgecolor="black",
                # palette=TOOLS_PALETTE,
                order=['MACS2', 'MACS2 broad', 'SPAN', 'SICER'],
                ax=ax)
    sns.swarmplot(data=df_peaks[df_peaks['modification'] == m], x='tool', y='peaks',  hue='dataset',
                  dodge=True, size=3, color="black", alpha=0.5, palette='dark:black',
                  order=['MACS2', 'MACS2 broad', 'SPAN', 'SICER'],
                  legend=False,
                  ax=ax)
    ax.xaxis.set_tick_params(rotation=90)
    ax.title.set_text(m)
    if i > 0:
        ax.set_ylabel(None)
    else:
        ax.set_ylabel('peaks')
    if i < 4:
        ax.legend().set_visible(False)
    else:
        # Put a legend to the right of the current axis
        ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

    # ax.set_ylim(0, 1.9e7)
plt.tight_layout()
# plt.savefig(GSE26320_PATH_HG38 + f'/pics/library_mapped_detailed_all.pdf', bbox_inches='tight', dpi=300)
plt.show()

## Compute BAM coverage

In [ ]:
def load_bams(path, modifications, cells, replicates, control='Input'):
    df_bams = pd.DataFrame(columns=['bam_file', 'modification', 'cell', 'replicate', 'reads'], dtype=object)
    for f in tqdm(os.listdir(path)):
        if not f.endswith('.bam'):
            continue
        cell = next((c for c in cells if c in f), None)
        rep = next((r for r in replicates if r in f), None)
        mod = next((m for m in modifications if m in f), None)
        if mod is None and control is not None and control in f:
            mod = 'Control'
        if cell and mod and rep is not None:
            # # Limit cells
            # if cell not in ['GM12878', 'K562', 'Huvec', 'TCell', 'BCell', 'Monocyte']:
            #     continue
            file = os.path.join(path, f)
            reads = ! samtools view -F 0x04 -c {file}
            reads = int(reads[0])
            df_bams.loc[len(df_bams)] = (file, mod, cell, rep, reads)
    return df_bams


In [ ]:
df_encode_bams = load_bams(GSE26320_PATH + '/bams', MODIFICATIONS, GSE26320_CELLS, GSE26320_REPS, 'Input')
df_encode_bams['dataset'] = 'ENCODE'
df_encode_bams

In [ ]:
df_immune_bams = load_bams(IMMUNE_PATH + '/bams', MODIFICATIONS, IMMUNE_CELLS, IMMUNE_REPS, 'Control')
df_immune_bams['dataset'] = 'Roadmap'
df_immune_bams

In [ ]:
df_bams = pd.concat([df_encode_bams, df_immune_bams]).reset_index(drop=True)
df_bams = df_bams.loc[[(m, c, r) not in OUTLIERS for _, (m, c, r) in df_bams[['modification', 'cell', 'replicate']].iterrows()]].copy()

In [ ]:
plt.figure(figsize=(6, 4))
ax = plt.axes()
g_results = sns.barplot(data=df_bams,
                        hue='modification', x='dataset', y='reads', ax=ax,
                        capsize=.05, err_kws={'linewidth': 2}, edgecolor="black",
                        hue_order=MODIFICATIONS+['Control']
                        )
ax.set_title('Library size')
# Put a legend to the right of the current axis
g_results.axes.legend(loc='center left', bbox_to_anchor=(1, 0.5))
ax.xaxis.set_tick_params(rotation=90)

plt.tight_layout()
plt.show()

# Find control tracks

In [ ]:
! mkdir -p {DOWNSAMPLE_DIR}/bams
controls = []
control_reads = []

for _, (ds, m, c, r) in tqdm(
        df_bams[['dataset', 'modification', 'cell', 'replicate']].iterrows()):
    if m == 'Control':
        controls.append(None)
        control_reads.append(None)
        continue
    # # Limit cells
    # if c not in ['GM12878', 'K562', 'Huvec', 'TCell', 'BCell', 'Monocyte']:
    #     controls.append(None)
    #     control_reads.append(None)
    #     continue
    print(ds, m, c, r)
    t = df_bams[(df_bams['dataset'] == ds) & (df_bams['modification'] == 'Control') & (df_bams['cell'] == c)]
    tr = t[t['replicate'] == r]
    bam, reads = None, None
    if len(tr) == 1:
        bam = tr['bam_file'].values[0]
        reads = tr['reads'].values[0]
    elif len(t) == 1:
        bam = t['bam_file'].values[0]
        reads = t['reads'].values[0]
    controls.append(bam)
    control_reads.append(reads)
    if bam is None:
        print('No control', ds, m, c, r)
        continue

df_bams['control_file'] = controls
df_bams['control_reads'] = control_reads
df_bams.sample(3)

# Mix control
Use chromosome 1 for speed purposes.

In [ ]:
MIX_DIR = os.path.expanduser('~/data/2023_mix_chr1')
MIX_SIGNAL_FRACTIONS = [0.7, 0.5, 0.2, 0.1]

In [ ]:
# ! mkdir -p {MIX_DIR}/bams
# 
# tf = tempfile.mktemp()
# 
# mixtures = []
# for _, (ds, m, c, r, bam_file, control_file, reads, control_reads) in tqdm(
#         df_bams[df_bams['modification'] != 'Control'][
#             ['dataset', 'modification', 'cell', 'replicate', 'bam_file', 'control_file', 'reads', 'control_reads']
#         ].iterrows()):
#     # Limit cells
#     if c not in ['GM12878', 'K562', 'Huvec', 'TCell', 'BCell', 'Monocyte']:
#         continue    
#     print(ds, m, c, r)
#     control_name = os.path.basename(control_file)
#     ! ln -sf {control_file} {MIX_DIR}/bams/{control_name}
#     
#     for fraction in MIX_SIGNAL_FRACTIONS:
#         print(fraction)
#         result_file = f'{MIX_DIR}/bams/{m}_{c}_{r}_{fraction}.bam'
#         print(result_file)
#         mixtures.append((ds, m, c, r, fraction, result_file))
#         if os.path.exists(result_file):
#             continue
#         control_fraction = min(1.0, (1.0 - fraction) * reads / control_reads)
#         ! samtools view -@ 4 -H {bam_file} > {tf}.sam
#         ! samtools view -@ 4 -s {fraction} {bam_file} >> {tf}.sam
#         ! samtools view -@ 4 -s {control_fraction} {control_file} >> {tf}.sam
#         ! samtools view -S -b {tf}.sam > {tf}.bam
#         ! samtools sort {tf}.bam -o {result_file}
#         ! rm {tf}.sam {tf}.bam
#     
# df_mixtures = pd.DataFrame(mixtures, columns=['dataset', 'modification', 'cell', 'replicate', 'fraction', 'bam_file'])
# del mixtures
# df_mixtures.sample(3)        

In [ ]:
! mkdir -p {MIX_DIR}/bams

tf = tempfile.mktemp()
tab = '\t'

mixtures = []
for _, (ds, m, c, r, bam_file, control_file, reads, control_reads) in tqdm(list(
        df_bams[df_bams['modification'] != 'Control'][
            ['dataset', 'modification', 'cell', 'replicate', 'bam_file', 'control_file', 'reads', 'control_reads']
        ].iterrows())):
    # # Limit cells
    # if c not in ['GM12878', 'K562', 'Huvec', 'TCell', 'BCell', 'Monocyte']:
    #     continue
    print(ds, m, c, r)
    control_name = os.path.basename(control_file)
    if not os.path.exists(f'{MIX_DIR}/bams/{control_name}'):
        print('Control')
        ! samtools view -@ 4 -H {control_file} > {tf}.sam
        ! samtools view -@ 4 {control_file} | grep "chr1{tab}" >> {tf}.sam
        ! samtools view -S -b {tf}.sam > {tf}.bam
        ! samtools sort {tf}.bam -o {MIX_DIR}/bams/{control_name}
        ! rm {tf}.sam {tf}.bam
        # ! ln -sf {control_file} {MIX_DIR}/bams/{control_name}

    for fraction in MIX_SIGNAL_FRACTIONS:
        print(fraction)
        result_file = f'{MIX_DIR}/bams/{m}_{c}_{r}_{fraction}.bam'
        print(result_file)
        mixtures.append((ds, m, c, r, fraction, result_file))
        if os.path.exists(result_file):
            continue
        control_fraction = min(1.0, (1.0 - fraction) * reads / control_reads)
        ! samtools view -@ 4 -H {bam_file} > {tf}.sam
        ! samtools view -@ 4 -s {fraction} {bam_file} | grep "chr1{tab}" >> {tf}.sam
        ! samtools view -@ 4 -s {control_fraction} {control_file} | grep "chr1{tab}" >> {tf}.sam
        ! samtools view -S -b {tf}.sam > {tf}.bam
        ! samtools sort {tf}.bam -o {result_file}
        ! rm {tf}.sam {tf}.bam

df_mixtures = pd.DataFrame(mixtures, columns=['dataset', 'modification', 'cell', 'replicate', 'fraction', 'bam_file'])
del mixtures
df_mixtures.sample(3)

## Peak calling and visualization
Peak calling SPAN: important to keep fixed SPAN settings.

```
cd ~/data/2023_mi
conda activate snakemake
snakemake -p -s ~/work/chipseq-smk-pipeline/Snakefile all --cores all  --use-conda  --directory $(pwd) --config genome=hg38 fastq_dir=$(pwd) start_with_bams=True bams_dir=bams macs2=True span=False sicer=False --rerun-trigger mtime;
snakemake -p -s ~/work/chipseq-smk-pipeline/Snakefile all --cores all  --use-conda  --directory $(pwd) --config genome=hg38 fastq_dir=$(pwd) start_with_bams=True bams_dir=bams macs2=True macs2_mode=broad macs2_params="--broad --broad-cutoff=0.1" macs2_suffix="broad0.1"  span=False sicer=False --rerun-trigger mtime; 
snakemake -p -s ~/work/chipseq-smk-pipeline/Snakefile all --cores all  --use-conda  --directory $(pwd) --config genome=hg38 fastq_dir=$(pwd) start_with_bams=True bams_dir=bams bamCoverage_params="--ignoreDuplicates" span=True sicer=True span_bin=100 span_params="--keep-cache --noclip --debug  --blacklist /home/jetbrains/data/2023_wiggle/hg38-blacklist.v2.bed" --rerun-trigger mtime
```

# Analyze results

In [ ]:
! mkdir -p {MIX_DIR}/original_chr1

mixed_peaks_df = load_peaks_path(MIX_DIR, MODIFICATIONS, GSE26320_CELLS + IMMUNE_CELLS, GSE26320_REPS + IMMUNE_REPS)
mixed_peaks_df['dataset'] = 'ENCODE'
mixed_peaks_df.loc[mixed_peaks_df['cell'].isin(IMMUNE_CELLS), 'dataset'] = 'Roadmap'
mixed_peaks_df['fraction'] = 0
for fraction in MIX_SIGNAL_FRACTIONS:
    mixed_peaks_df.loc[(mixed_peaks_df['file'].str.contains(f'_{fraction}_')) |
                       (mixed_peaks_df['file'].str.contains(f'_{fraction}-')), 'fraction'] = fraction
t = df_peaks[(df_peaks['cell'].isin(mixed_peaks_df['cell'].unique())) &
             (df_peaks['replicate'].isin(mixed_peaks_df['replicate'].unique()))][[
    'file', 'modification', 'cell', 'replicate', 'peaks', 'tool', 'dataset']].copy()
t['fraction'] = 1.0
# Processing original files
t = df_peaks[(df_peaks['cell'].isin(mixed_peaks_df['cell'].unique())) &
             (df_peaks['replicate'].isin(mixed_peaks_df['replicate'].unique()))][[
    'file', 'modification', 'cell', 'replicate', 'peaks', 'tool', 'dataset']].copy()
t['fraction'] = 1.0
chr1_files = []
chr1_peaks = []
for file in tqdm(t['file']):
    chr1_file = f'{MIX_DIR}/original_chr1/{os.path.basename(file)}'
    if not os.path.exists(chr1_file):
        tab = '\t'
        ! cat {file} | grep "chr1{tab}" > {chr1_file}
    chr1_files.append(chr1_file)
    chr1_peaks.append(lines(chr1_file))
t['file'] = chr1_files
t['peaks'] = chr1_peaks
mixed_peaks_df = pd.concat([mixed_peaks_df, t]).reset_index(drop=True)
mixed_peaks_df.sample(3)

## Peaks number

In [ ]:
def plot_modified(df, y, line):
    for ds in ['ENCODE', 'Roadmap']:
        print(ds)
        plt.figure(figsize=(15, 3))
        axs = [plt.subplot(1, 5, i + 1) for i in range(5)]

        for i, m in enumerate(MODIFICATIONS):
            ax = axs[i]
            t = df[(df['dataset'] == ds) & (df['modification'] == m)].copy()
            fractions = sorted(df['fraction'].unique(), reverse=True)
            t['j'] = 0
            for j, fraction in enumerate(fractions):
                t.loc[t['fraction'] == fraction, 'j'] = j
            if line:
                sns.lineplot(data=t,
                             x='j', y=y, hue='tool',
                             hue_order=['MACS2', 'MACS2 broad', 'SICER', 'SPAN'],
                             palette=TOOLS_PALETTE,
                             ax=ax,
                             errorbar='se', marker='o', markersize=10
                             )
                ax.set_xticks(range(len(fractions)), minor=False)
                ax.set_xticklabels([str(f) for f in fractions])
                ax.set_xlabel('fraction')
                # We could use just invert axis, but this won't make linear dots
                # ax.invert_xaxis()
            else:
                sns.barplot(data=t,
                                    x='tool', y=y, hue='fraction', ax=ax,
                                    capsize=.1, err_kws={'linewidth': 2}, edgecolor="black",
                                    order=['MACS2', 'MACS2 broad', 'SPAN', 'SICER'],
                                    hue_order = sorted(df['fraction'].unique(), reverse=True),
                                    )
            ax.xaxis.set_tick_params(rotation=90)
            ax.title.set_text(m)
            if i > 0:
                ax.set_ylabel(None)
            if i < len(axs) - 1:
                ax.legend().set_visible(False)
            else:
                # Put a legend to the right of the current axis
                ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))    

        plt.tight_layout()
        plt.show()

In [ ]:
print('Mixed')
plot_modified(mixed_peaks_df, 'peaks', False)

In [ ]:
# mixed_peaks_df[mixed_peaks_df['tool'] == 'SPAN']

## Jaccard vs original peaks

In [ ]:
import pyranges as pr

def overlap_vs_original(df):
    overlaps = []
    for _, (ds, m, c, r, fraction, tool, file) in tqdm(list(df[df['fraction'] != 1.0][[
        'dataset', 'modification', 'cell', 'replicate', 'fraction', 'tool', 'file'
    ]].iterrows())):
        t = df[(df['fraction'] == 1.0) &
               (df['dataset'] == ds) &
               (df['modification'] == m) &
               (df['cell'] == c) &
               (df['replicate'] == r) &
               (df['tool'] == tool)]
        if len(t) != 1:
            print('No original peaks for', ds, m, c, r, tool)
            continue
        original_file = t['file'].values[0]
        peaks = lines(file)
        original_peaks = lines(original_file)
        if peaks == 0 or original_peaks == 0:
            peaks_len = int(bedl(file).sum())
            original_peaks_len = int(bedl(original_file).sum())
            overlaps.append((ds, m, c, r, fraction, tool,
                             peaks, peaks_len, original_peaks, original_peaks_len,
                             0, 0, 0, 0))
            continue
        file = pr.read_bed(file)
        original_file = pr.read_bed(original_file)
        original_peaks_len = original_file.lengths().sum()
        peaks_len = file.lengths().sum()
        # !bedtools intersect -a {original_file} -b {file} -wa -u > {tf}
        original_peaks_overlap = len(original_file.overlap(file))
        # !bedtools intersect -b {original_file} -a {file} -wa -u > {tf}
        peaks_overlap = len(file.overlap(original_file))
        # !bedtools intersect -a {original_file} -b {file} -wo > {tf}
        overlap_len = file.intersect(original_file).lengths().sum()
        overlaps.append((ds, m, c, r, fraction, tool, 
                         peaks, peaks_len, original_peaks, original_peaks_len,
                         peaks_overlap, original_peaks_overlap, overlap_len, 
                         d(overlap_len, peaks_len + original_peaks_len - overlap_len)))
        
    return pd.DataFrame(overlaps, columns=[
        'dataset', 'modification', 'cell', 'replicate', 'fraction', 'tool', 
        'peaks', 'peaks_len', 'original_peaks', 'original_peaks_len',
        'peaks_overlap', 'original_peaks_overlap', 'overlap_len', 'jaccard'
    ])

In [ ]:
mixed_peaks_overlap_df = overlap_vs_original(mixed_peaks_df)
mixed_peaks_overlap_df.sample(3)

In [ ]:
mixed_peaks_overlap_df[mixed_peaks_overlap_df['tool'] == 'SPAN']

In [ ]:
# Add 1.0 and 0.0 fraction
for _, (ds, m, c, r, t) in list(
        mixed_peaks_overlap_df[
            ['dataset', 'modification', 'cell', 'replicate', 'tool']].drop_duplicates().iterrows()):
    mixed_peaks_overlap_df.loc[len(mixed_peaks_overlap_df)] = \
        [ds, m, c, r, 0.0, t] + [None] * (len(mixed_peaks_overlap_df.columns) - 7) + [0.0]
    mixed_peaks_overlap_df.loc[len(mixed_peaks_overlap_df)] = \
        [ds, m, c, r, 1.0, t] + [None] * (len(mixed_peaks_overlap_df.columns) - 7) + [1.0]
mixed_peaks_overlap_df.sample(3)

In [ ]:
from datetime import datetime
now = datetime.now().isoformat(timespec='minutes')
rfile = f'{MIX_DIR}/mixed_peaks_overlap_df_{now}.csv.gz'
print(rfile)
mixed_peaks_overlap_df.to_csv(rfile, compression='gzip', index=None)

In [ ]:
print('Mixed control')
plot_modified(mixed_peaks_overlap_df, 'jaccard', True)

In [ ]:
print('ENCODE')
plt.figure(figsize=(4, 4))
t = mixed_peaks_overlap_df[
    (mixed_peaks_overlap_df['dataset']=='ENCODE') &
    (mixed_peaks_overlap_df['modification']=='H3K4me1')
    ].copy()
t.sort_values(by=['fraction'], ascending=False, inplace=True)
t['quality'] = [f'{int(f * 100)}%' for f in t['fraction']]
sns.lineplot(data=t, x="quality", y="jaccard", hue='tool',
             hue_order=['MACS2', 'MACS2 broad', 'SPAN', 'SICER'],
             palette=TOOLS_PALETTE,
             # errorbar=('sd', .5), 
             errorbar='se', 
             marker='o', markersize=8)
plt.title('Jaccard vs original peaks')
plt.xlabel('Quality')
plt.ylabel('Jaccard')
plt.tight_layout()
plt.show()

In [ ]:
print('Immune')
plt.figure(figsize=(4, 4))
ax = plt.axes()
t = mixed_peaks_overlap_df[
    (mixed_peaks_overlap_df['dataset']=='Roadmap') &
    (mixed_peaks_overlap_df['modification']=='H3K4me1')
    ].copy()
t.sort_values(by=['fraction'], ascending=False, inplace=True)
t['quality'] = [f'{int(f * 100)}%' for f in t['fraction']]
sns.lineplot(data=t[t['tool'] != 'SPAN'], x="quality", y="jaccard", hue='tool',
             hue_order=['MACS2', 'MACS2 broad', 'SPAN', 'SICER'],
             palette=TOOLS_PALETTE,
             errorbar='se',
             marker='o', markersize=8,
             ax=ax)
sns.lineplot(data=t[t['tool'] == 'SPAN'], x="quality", y="jaccard", hue='tool',
             hue_order=['MACS2', 'MACS2 broad', 'SPAN', 'SICER'],
             palette=TOOLS_PALETTE,
             errorbar='se',
             marker='o', markersize=8,
             legend=False,
             ax=ax)

plt.title('Jaccard vs original peaks')
plt.xlabel('Quality')
plt.ylabel('Jaccard')
plt.tight_layout()
plt.show()

In [ ]:
def compute_auc(df, x):
    rows = []
    for (ds, m, c, r, tool), t in df.groupby(['dataset', 'modification', 'cell', 'replicate', 'tool']):
        t = t.copy()
        t.sort_values(by=['fraction'], inplace=True)
        auc = 0
        fraction_prev = 0
        value_prev = 0
        for _, (fraction, value) in t[['fraction', x]].iterrows():
            auc += (fraction - fraction_prev) * (value + value_prev) / 2
            fraction_prev, value_prev = fraction, value
        rows.append((ds, m, c, r, tool, auc))
    return pd.DataFrame(data=rows, columns=['dataset', 'modification', 'cell', 'replicate', 'tool', 'auc'])

In [ ]:
mixed_peaks_overlap_auc_df = compute_auc(mixed_peaks_overlap_df, 'jaccard')
mixed_peaks_overlap_auc_df.sample(3)

In [ ]:
for ds in ['ENCODE', 'Roadmap']:
    print(ds)
    plt.figure(figsize=(6, 3))
    ax = plt.axes()
    ax.title.set_text('AUC jaccard vs original peaks')
    sns.barplot(data=mixed_peaks_overlap_auc_df[mixed_peaks_overlap_auc_df['dataset']==ds], 
                x='modification', y='auc',
                hue='tool',
                capsize=.05, err_kws={'linewidth': 2},
                order=MODIFICATIONS,
                hue_order=['MACS2', 'MACS2 broad', 'SPAN', 'SICER'],
                ax=ax,
                )
    sns.swarmplot(data=mixed_peaks_overlap_auc_df[mixed_peaks_overlap_auc_df['dataset']==ds],
                  x='modification', y='auc',
                  hue='tool',
                  dodge=True, size=2, color="black", alpha=0.5, palette='dark:black',
                  order=MODIFICATIONS,
                  hue_order=['MACS2', 'MACS2 broad', 'SPAN', 'SICER'],
                  legend=False,
                  ax=ax)
    ax.xaxis.set_tick_params(rotation=45)
    # Put a legend to the right of the current axis
    ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    plt.tight_layout()
    plt.show()

In [ ]:
print('ENCODE + Roadmap')
plt.figure(figsize=(6, 3))
ax = plt.axes()
ax.title.set_text('AUC jaccard vs original peaks')
sns.barplot(data=mixed_peaks_overlap_auc_df,
            x='modification', y='auc',
            hue='tool',
            capsize=.2, err_kws={'linewidth': 2},
            order=MODIFICATIONS,
            hue_order=['MACS2', 'MACS2 broad', 'SPAN', 'SICER'],
            ax=ax,
            )
sns.swarmplot(data=mixed_peaks_overlap_auc_df,
              x='modification', y='auc',
              hue='tool',
              dodge=True, size=1.5, color="black", alpha=0.5, palette='dark:black',
              order=MODIFICATIONS,
              hue_order=['MACS2', 'MACS2 broad', 'SPAN', 'SICER'],
              legend=False,
              ax=ax)
ax.xaxis.set_tick_params(rotation=45)
# Put a legend to the right of the current axis
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.tight_layout()
plt.show()

## Jaccard between replicates

In [ ]:
def compute_reps_overlap(df_peaks, modifications, cells):
    reps_overlap = pd.DataFrame(columns=['dataset', 'modification', 'cell', 'fraction', 'tool', 'rep1', 'rep2',
                                         'peaks1', 'peaks1_len', 'peaks2', 'peaks2_len',
                                         'peaks1_overlap', 'peaks2_overlap', 'peaks_overlap_len'], dtype=object)

    tools = list(sorted(set(df_peaks['tool'])))
    for ds, m, c, f in tqdm(list(product(['ENCODE', 'Roadmap'], modifications, cells, MIX_SIGNAL_FRACTIONS + [1.0]))):
        print(ds, m, c, f)
        tm = df_peaks[(df_peaks['dataset'] == ds) & (df_peaks['modification'] == m) & (df_peaks['cell'] == c) & (df_peaks['fraction'] == f)]
        reps = list(sorted(set(tm['replicate'])))
        for tool in tools:
            # print(m, tool, len(reps))
            files = [None] * len(reps)
            peaks = [-1] * len(reps)
            peaks_lens = [-1] * len(reps)
            for i in range(len(reps)):
                t1 = tm[(tm['tool'] == tool) & (tm['replicate'] == reps[i])]
                if len(t1) == 0:
                    continue
                file = t1['file'].values[0]
                peaks[i] = lines(file)
                if peaks[i] == 0:
                    peaks_lens[i] = 0
                    continue
                files[i] = pr.read_bed(file)
                peaks_lens[i] = files[i].lengths().sum()
            for i, j in product(range(len(reps)), range(len(reps))):
                if i >= j:
                    continue
                rep1, rep2 = reps[i], reps[j]
                file1, file2 = files[i], files[j]
                peaks1, peaks2 = peaks[i], peaks[j]
                peaks1_len, peaks2_len = peaks_lens[i], peaks_lens[j]
                if peaks1 <= 0 or peaks2 <= 0:
                    reps_overlap.loc[len(reps_overlap)] = \
                        (ds, m, c, f, tool, rep1, rep2, peaks1, peaks1_len, peaks2, peaks2_len,
                         0, 0, 0)
                    continue
                overlap1 = len(file1.overlap(file2))
                overlap2 = len(file2.overlap(file1))
                overlap_len = file1.intersect(file2).lengths().sum()
                reps_overlap.loc[len(reps_overlap)] = \
                    (ds, m, c, f, tool, rep1, rep2, peaks1, peaks1_len, peaks2, peaks2_len, 
                     overlap1, overlap2, overlap_len)
    reps_overlap['jaccard'] = [
        d(lo, l1 + l2 - lo)
        for l1, l2, lo in zip(reps_overlap['peaks1_len'], reps_overlap['peaks2_len'], reps_overlap['peaks_overlap_len'])
    ]
    return reps_overlap


In [ ]:
df_reps_overlap = compute_reps_overlap(mixed_peaks_df, MODIFICATIONS, GSE26320_CELLS + IMMUNE_CELLS)
# Add 0.0 fraction
for _, (ds, m, c, t) in list(
        df_reps_overlap[
            ['dataset', 'modification', 'cell', 'tool']].drop_duplicates().iterrows()):
    df_reps_overlap.loc[len(df_reps_overlap)] = \
        [ds, m, c, 0.0, t] + [None] * (len(df_reps_overlap.columns) - 6) + [0.0]
df_reps_overlap.sample(5)

In [ ]:
mixed_peaks_df[(mixed_peaks_df['dataset'] == 'Roadmap') & 
               (mixed_peaks_df['modification'] == 'H3K27ac')][['cell', 'replicate']].drop_duplicates()

In [ ]:
print('Mixed control replicates overlap')
plot_modified(df_reps_overlap, 'jaccard', True)

In [ ]:
print('ENCODE')
plt.figure(figsize=(4, 4))
t = df_reps_overlap[
    (df_reps_overlap['dataset']=='ENCODE') &
    (df_reps_overlap['modification']=='H3K27ac') 
    ].copy()
t.sort_values(by=['fraction'], ascending=False, inplace=True)
t['quality'] = [f'{int(f * 100)}%' for f in t['fraction']]
sns.lineplot(data=t, x="quality", y="jaccard", hue='tool',
             hue_order=['MACS2', 'MACS2 broad', 'SPAN', 'SICER'],
             palette=TOOLS_PALETTE,
             # errorbar=('sd', .5),
             errorbar='se', 
             marker='o', markersize=8)
plt.title('Jaccard between replicates')
plt.xlabel('Quality')
plt.ylabel('Jaccard')
plt.tight_layout()
plt.show()

In [ ]:
print('Immune')
plt.figure(figsize=(4, 4))
t = df_reps_overlap[
    (df_reps_overlap['dataset']=='Roadmap') &
    (df_reps_overlap['modification']=='H3K4me1')
    ].copy()
t.sort_values(by=['fraction'], ascending=False, inplace=True)
t['quality'] = [f'{int(f * 100)}%' for f in t['fraction']]
sns.lineplot(data=t, x="quality", y="jaccard", hue='tool',
             hue_order=['MACS2', 'MACS2 broad', 'SPAN', 'SICER'],
             palette=TOOLS_PALETTE,
             # errorbar=('sd', .5),
             errorbar='se', 
             marker='o', markersize=8)
plt.title('Jaccard between replicates')
plt.xlabel('Quality')
plt.ylabel('Jaccard')
plt.tight_layout()
plt.show()

In [ ]:
def compute_auc_reps(df, x):
    rows = []
    for (ds, m, c, tool), t in df.groupby(['dataset', 'modification', 'cell', 'tool']):
        t = t.copy()
        t.sort_values(by=['fraction'], inplace=True)
        auc = 0
        fraction_prev = 0
        value_prev = 0
        for _, (fraction, value) in t[['fraction', x]].iterrows():
            auc += (fraction - fraction_prev) * (value + value_prev) / 2
            fraction_prev, value_prev = fraction, value
        rows.append((ds, m, c, tool, auc))
    return pd.DataFrame(data=rows, columns=['dataset', 'modification', 'cell', 'tool', 'auc'])

In [ ]:
df_reps_overlap_auc_df = compute_auc_reps(df_reps_overlap, 'jaccard')
df_reps_overlap_auc_df.sample(3)

In [ ]:
for ds in ['ENCODE', 'Roadmap']:
    print(ds)
    plt.figure(figsize=(6, 3))
    ax = plt.axes()
    ax.title.set_text('AUC jaccard between replicates')
    sns.barplot(data=df_reps_overlap_auc_df[df_reps_overlap_auc_df['dataset']==ds],
                x='modification', y='auc',
                hue='tool',
                capsize=.05, err_kws={'linewidth': 2},
                order=MODIFICATIONS,
                hue_order=['MACS2', 'MACS2 broad', 'SPAN', 'SICER'],
                ax=ax,
                )
    sns.swarmplot(data=df_reps_overlap_auc_df[df_reps_overlap_auc_df['dataset']==ds],
                  x='modification', y='auc',
                  hue='tool',
                  dodge=True, size=2, color="black", alpha=0.5, palette='dark:black',
                  order=MODIFICATIONS,
                  hue_order=['MACS2', 'MACS2 broad', 'SPAN', 'SICER'],
                  legend=False,
                  ax=ax)

    ax.xaxis.set_tick_params(rotation=45)
    # Put a legend to the right of the current axis
    ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    plt.tight_layout()
    plt.show()

In [ ]:
print('ENCODE + Roadmap')
plt.figure(figsize=(6, 3))
ax = plt.axes()
ax.title.set_text('AUC jaccard between replicates')
sns.barplot(data=df_reps_overlap_auc_df,
            x='modification', y='auc',
            hue='tool',
            capsize=.2, err_kws={'linewidth': 2},
            order=MODIFICATIONS,
            hue_order=['MACS2', 'MACS2 broad', 'SPAN', 'SICER'],
            ax=ax,
            )
sns.swarmplot(data=df_reps_overlap_auc_df,
              x='modification', y='auc',
              hue='tool',
              dodge=True, size=2, color="black", alpha=0.5, palette='dark:black',
              order=MODIFICATIONS,
              hue_order=['MACS2', 'MACS2 broad', 'SPAN', 'SICER'],
              legend=False,
              ax=ax)

ax.xaxis.set_tick_params(rotation=45)
# Put a legend to the right of the current axis
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.tight_layout()
plt.show()